# First and Follow of Grammar

In [1]:
epsilon = '\u03B5'
arrow = ' \u2192 '
import pandas as pd

In [2]:
def printGrammar(Grammar):
    lhs = list(set([t[0] for t in Grammar]))
    for l in lhs:
        rhs = [t[1] for t in Grammar if t[0] == l]
        if '' in rhs :
            rhs.remove('')
            rhs.append(epsilon)
        b = ' | '.join(rhs)
        print('{0:<3} {2} {1:<25}'.format(l,b,arrow))

In [3]:
def First(X,G):
    first = []
    nt = list(set([t[0] for t in G]))
    if X == '':
        first.append(epsilon)
        return first
    
    for x in X: 
        prod = [t[1] for t in G if t[0]==x]
        
        if x not in nt:
            first.append(x)
            if epsilon in first:
                first.remove(epsilon)
            return first

        if '' in prod:
            first.append(epsilon)
            
        for p in prod:
            for i in p:
                if i not in nt:
                    first.append(i)
                    break
                else:
                    pr = [t[1] for t in G if t[0]==i]
                    first += First(i,G)
                    if '' in pr:
                        if i != p[-1]:
                            first.remove(epsilon)
                        continue
                    break
        if epsilon not in first:
            break
            
    return list(set(first))


In [4]:
def Follow(x,G,start):
    nt = list(set([t[0] for t in G]))
    follow = []
    if x not in nt:
        return -1
    if x == start:
        follow.append('$')
    for a,b in G:
        if x in b:
            spl = b.split(x,maxsplit=1)
            follow += First(spl[1],G)
            if epsilon in follow:
                follow.remove(epsilon)
            if epsilon in First(spl[1],G):
                follow += Follow(a,G,start)
    return list(set(follow))


In [5]:
G = [('S','ACB'),('S','Cbb'),('S','Ba'),('A','da'),('A','BC'),('B',''),('B','g'),('C',''),('C','h')]
nt = list(set([t[0] for t in G]))
start = 'S'
print("\nGrammar : \n")
printGrammar(G)
for n in nt:
    print("\nFirst({0}) = {1}".format(n,First(n,G)))
print("\n")
for n in nt:
    print("\nFollow({0}) = {1}".format(n,Follow(n,G,start)))


Grammar : 

A    →  da | BC                  
B    →  g | ε                    
C    →  h | ε                    
S    →  ACB | Cbb | Ba           

First(A) = ['h', 'g', 'd', 'ε']

First(B) = ['ε', 'g']

First(C) = ['ε', 'h']

First(S) = ['ε', 'a', 'g', 'h', 'd', 'b']



Follow(A) = ['$', 'g', 'h']

Follow(B) = ['$', 'g', 'a', 'h']

Follow(C) = ['$', 'g', 'h', 'b']

Follow(S) = ['$']


# Creating Parsing Table

In [39]:
def ParsingTable(G):
    '''
    Input : Grammar
    Output: None with a message if G is not LL(1) else the parsing table
    '''
    ll = True
    nt = list(set([t[0] for t in G]))
    ter = list(set([t[1] for t in G if t[1] not in nt ]))
    t = []
    for i in ter:
        if i != '':
            for j in i:
                if j not in nt:
                    t.append(j)
    t.append('$')
    t = list(set(t))
    pt = pd.DataFrame(index=nt, columns=t)
    pt = pt.fillna('')
    for i in pt.columns:
        pt[i] = pt[i].apply(list)
    for a,b in G:
        fir = First(b,G)
        for i in fir:
            if i != epsilon:
                pt.loc[a,i].append(a + arrow + (b if b != '' else epsilon))
        if epsilon in fir:
            fol = Follow(a,G,start)
            for i in fol:
                pt.loc[a,i].append(a + arrow + (b if b != '' else epsilon))
    for i in pt.columns:
        pt[i] = pt[i].apply(lambda x : list(set(x)))
        pt[i] = pt[i].apply(lambda x : '-' if len(x)==0 else ' | '.join(x))
    
    conflict = False
    for i in pt.columns:
        conflict = conflict or (pt[i].apply(lambda x : True if '|' in x else False).any())
    if not conflict:
        return pt
    else:
        print("\n\nGrammar is not LL(1)")
        return

In [36]:
G = [('S','ACB'),('S','Cbb'),('S','Ba'),('A','da'),('A','BC'),('B',''),('B','g'),('C',''),('C','h')]
start = 'S'
print("\nGrammar : \n")
printGrammar(G)
ParsingTable(G)


Grammar : 

A    →  da | BC                  
B    →  g | ε                    
C    →  h | ε                    
S    →  ACB | Cbb | Ba           


Grammar is not LL(1)


In [37]:
G = [('A','a'),('S','A'),('S','a')]
start = 'S'
print("\nGrammar : \n")
printGrammar(G)
ParsingTable(G)


Grammar : 

A    →  a                        
S    →  A | a                    


Grammar is not LL(1)


In [38]:
G = [('S','F'),('S','(S+F)'),('F','a')]
start = 'S'
print("\nGrammar : \n")
printGrammar(G)
ParsingTable(G)


Grammar : 

F    →  a                        
S    →  F | (S+F)                


,(,),a,$,+
F,-,-,F → a,-,-
S,S → (S+F),-,S → F,-,-
